<img src="https://raw.githubusercontent.com/ryanedw/COMPSS-202-SU24/main/Images/UCB-macss.jpg" width="120" align="right"/>
<h1>COMPSS 202 Class 14</h1>

<h2>Hypothesis Testing</h2>

Inspired by [SticiGui Chapter 27](https://www.stat.berkeley.edu/~stark/SticiGui/Text/testing.htm)

Is a ballot measure likely to pass if an opinion poll with 1,000 randomly drawn respondents shows a proportion $\hat{p} = 0.60$ of likely voters in favor of it? When people are asked about their height, do people say they are taller than they actually are? Are men more likely to overstate their height than women?

These are all examples of hypotheses that we can test with samples.


<h3>Learning objectives:</h3>

<ol style="margin-top: 0; margin-bottom: 0;">
  <li>Standard errors in estimators like $\bar{X}$ imply confidence intervals
      <ul style="margin-top: 0; margin-bottom: 0;">
      <li>Two SE’s on either side of $\bar{X}$ picks up 95% of the distribution of the true population mean $\mu$
      </li>
    </ul>
  </li>
  <li>Confidence intervals imply hypothesis tests:
    <ul style="margin-top: 0; margin-bottom: 0;">
      <li>Think about what we care about, and reformulate it as a test that something equals zero, a proper <a href="https://en.wikipedia.org/wiki/Null_hypothesis">null hypothesis</a>
      </li>
      <li>Measure the average and the standard error (SE) of that something
      </li>
      <li>If the average is more than 2 SE’s away from zero, reject the null hypothesis that the something is zero, otherwise, fail to reject the null hypothesis
      </li>
    </ul>
  </li>
  <li>We can make Type-1 and Type-2 errors. Good statistical power reduces the change of Type-2 errors, false negatives that don’t detect
  </li>
</ol>

To begin, please run the cells below to load up the libraries necessary to access data in Google Sheets. Best practices include running the cells in order.

In [ ]:
install.packages("googlesheets4")
library(googlesheets4)
gs4_deauth()

# Also load up the "car" package, for Companion to Applied Regression
install.packages("car")
library(car)

The code below loads in data on gender and subjective and objective height for $n = 5,068$ respondents in wave 4 of the [National Longitudinal Study of Adolescent to Adult Health (Add Health)](https://addhealth.cpc.unc.edu/). Subjective or self-reported height in inches is measured by `r4height`, while objective height is `r4heightbio`.

<b>To view the Sheets file, click here:</b> [addhealth_height_w4_nomiss.sheets](https://docs.google.com/spreadsheets/d/1uVN0_Wgz30nnng7Bt7H7xfYIqKL_g6W9ifLbF7wMK04/edit?usp=sharing)

In [ ]:
sheet_url = "https://docs.google.com/spreadsheets/d/1uVN0_Wgz30nnng7Bt7H7xfYIqKL_g6W9ifLbF7wMK04/edit?usp=sharing"

ah_heights <- read_sheet(sheet_url,
                         range = "A1:F5069")

First let's quickly examine the top of the dataset, examine its dimensions, and assign $n$ to be the number of rows, which is the sample size.

In [ ]:
head(ah_heights)
dimensions = dim(ah_heights)
n = dimensions[1]
n

As you can see, `diff` is already programmed as the difference between `r4height`, the subjective measure that is self-reported, and `r4heightbio`, the objective measure that is collected by the interviewer. If the difference were not already in the dataset, we could have created it this way:

In [ ]:
ah_heights$diff0 = ah_heights$r4height - ah_heights$r4heightbio
head(ah_heights)

---

<h2>1. Examining by hand</h2>

Suppose our null hypothesis is that there is no difference between self-reported and objective height, and thus the difference between them is zero. `diff = 0`. Here is the sample average of the difference:

In [ ]:
mean_diff = mean(ah_heights$diff)
mean_diff

mean_diff = mean(ah_heights$r4height) - mean(ah_heights$r4heightbio)
mean_diff

In the code above, we also see that the average difference also equals the difference in the averages, up to floating point rounding error anyway! This is generally true about the sample average and the expectation: they are distributive across addition and subtraction.

In words, our results show that the average difference between self-reported height and objective height in these data is 0.42 inch. On average, people overstate how tall they are by four tenths of an inch. 

$$
\bar{Y} = \bar{X}^S - \bar{X}^O = 0.42
$$

If the Add Health sample was constructed using random sampling, we know the sample average is an unbiased measure of the true population mean. But how certain are we about the point estimate of 0.42 inch? Is it significantly different from zero? To proceed, construct the <b>standard error</b> of the sample mean.

The standard error of the mean of $Y$ equals the sample standard deviation of $Y$ divided by the square root of the sample size, $n$:

$$
SE \left[ 
\bar{Y}
\right]
=
\frac{s}{\sqrt{n}}
$$
where
$$
s = \sqrt{
\frac{1}{n-1} \sum_{i=1}^n \left(
Y_i - \bar{Y}
\right)^2
}
$$

In [ ]:
stderr_meandiff = sd(ah_heights$diff) / sqrt(n)
stderr_meandiff

stderr_meandiff = ( sum( (ah_heights$diff - mean_diff)^2 ) / (n-1) )^0.5 / sqrt(n)
stderr_meandiff

The first calculation method above uses `sd()`, the sample standard deviation routine. The second method writes out the sample standard deviation by hand, just to keep us honest. The two methods produce the same result for the $SE$. 

Finally, a 95% confidence interval around the mean, $\bar{X}$, is defined by roughly $\pm 2 \ SE$. Statistical programs will often use $\pm 1.96 \ SE$ in large enough samples, because $1.96$ is a critical value of [Student's $t$ distribution](https://en.wikipedia.org/wiki/Student%27s_t-distribution).

In [ ]:
mean_diff_conf = c(mean_diff - 2*stderr_meandiff, 
                   mean_diff + 2*stderr_meandiff)
mean_diff_conf

In words: the sample average of the difference between self-reported and objective height is 0.42 inches. This difference is statistically significantly different from zero at the 5% level, because zero lies outside of the 95% confidence interval defined by two standard errors on either side of the mean. The 95% confidence interval stretches roughly from 0.38 to 0.47.

<h2>2. Using ordinary least squares (OLS), the "big ladder"</h2>

While it's useful to see the long-form math at least once in your life, most of the time you will probably want to cut to the chase and get on with the social science! It turns out that estimating simple equations with OLS, which you can do using the built-in `lm()` routine (for "linear model") in __R__, will return the answers to simple questions.

Let's estimate this simple model:
$$
Y_i = \alpha + \epsilon_i
$$
where $Y_i$ is the difference between subjective and objective heights, $\alpha$ is a constant term whose estimate will end up being the sample average $\bar{Y}$, and $\epsilon_i$ is a white-noise error with mean zero. 

To get __R__ to estimate this function, we have to pass it a vector of 1's as the lone $x$-variable, and then __R__ will understand what we want. The syntax for `lm()` requires the first element to be the linear estimation equation, with a tilde where the equals sign would go, followed by options like a reference to the data frame storing the variables in columns.

A reasonable approach is to create an object like `reg1` and then pass it to `summary()` to examine results. A call to `confint()` produces a matrix with 95% confidence intervals around the `lm()` estimates.

In [ ]:
reg1 <- lm(diff ~ 1, data = ah_heights)
summary(reg1)
confint(reg1)

As you can see above, OLS returns an estimate of the constant term $\alpha$, called "(Intercept)", and its standard error. These are the same values we saw when calculating the sample average and its standard error by hand.

---

<h2>3. Do men and women respond differently?</h2>

Ordinary least squares also allows us to examine differences in $y$-variables across subgroups very quickly. Based on the [Data Collection Instrument and User Guide](https://adatawinter.site.wesleyan.edu/files/2017/08/AddHealth-Wave_1_Questionnaire-and-Codebook.pdf), I think Add Health probably began with "biological sex" as recorded for the person by the school, which likely drew from a parental report, and then asked interviewers to verify the measure by asking the respondent as subjectively needed. To what extent the variable measures gender versus sex seems like an open question. It appears that [NIH funded a companion study in 2017](https://addhealth.cpc.unc.edu/news/add-health-deputy-director-to-co-lead-new-nih-study/) to look at gender identity over the life course more closely.

The variable that we have indexes males as `r4sex == 1` and females as `r4sex == 2`. One actually could just use this variable in a linear model, but that would change the interpretation of the constant term $\alpha$. A better approach is to code a binary 0/1 indicator variable that measures group identity. Here, we can do that quickly by subtracting 1 from the "sex" variable:

In [ ]:
ah_heights$female = (ah_heights$r4sex - 1)

With this new `female` variable, we can run a simple OLS model that tests for differences between females and the reference group:

$$
Y_i = \alpha' + \beta \ female_i + \epsilon_i
$$

In these Add Health data, although one suspects that there probably is a spectrum of gender identity, the data are coded so that everyone who is not "female" is "male," and so the constant term $\alpha'$ tells us the average of $y$ among males. Generally speaking, the constant will measure the average among all groups that do not have an indicator variable of their own.

In [ ]:
reg2 <- lm(diff ~ female, data = ah_heights)
summary(reg2)
confint(reg2)

In words: males report being taller by about $\hat{\alpha}' = 0.7$ inch on average, and reported heights are $\hat{\beta} = -0.5$ inch more accurate on average among females. Both of those estimates are statistically significantly different from zero, because they lie (much) more than 2 of their standard errors away from 0.

<h2>4. Aligning the model with your question</h2>

In the example above, I found myself wanting to say something about the statistical significance of "excess" height reported by females. We recovered $\hat{\alpha}'$ and its standard error, which tells us about the average male response. But $\hat{\beta}$ tells us the average difference between men and women, not the average female response.

If we want to know about the average female response, we have some options:

<h3>4.1. Ask R to calculate the standard error of $\hat{\alpha}' + \beta$</h3>

The first line of code below calculates the sum of the two parameters from the `coef` column in the regression results that `lm()` created and placed in `reg2`. The second line of code asks __R__ to test whether that sum equals zero, and to do so, __R__ calculates the standard error of the sum (but doesn't report it). 

In [ ]:
alpha_plus_beta <- reg2$coef[1] + reg2$coef[2]
alpha_plus_beta

linearHypothesis(reg2, "(Intercept) + female = 0")

The results aren't exactly easy to read. The key parts are the __F__ and the __Pr(>F)__ columns. An $F$-statistic is kind of like a generalized $t$-statistic with units similar to the square of the $t$-statistic. Here, the $F$-stat is large, and thus the probability that the true value of $\alpha' + \beta = 0$ is very small. Thus we reject the null hypothesis that female self-reports of height are no different from objective height. Rather, they are about $\hat{\alpha}' + \hat{\beta} = 0.2$ inch taller, and that is statistically significantly different from zero.

---

<h3>4.2. Run the simple model on just one group</h3>

Another approach is to omit males from the data and estimate the simple model with no group indicator:

$$
Y^f_i = \alpha^f + \epsilon^f_i
$$

where I'm using the $f$ superscript to show that these estimates will be relevant only for the female subgroup. In __R__, we can estimate this model by passing a `subset()` of the dataframe with the `female == 1` conditional. 

In [ ]:
reg3f <- lm(diff ~ 1, 
            data = subset(ah_heights, female == 1)
            )
summary(reg3f)
confint(reg3f)

In words, we see the same story we just saw earlier: women state they are about $\hat{\alpha}^f = 0.2$ inch taller than they actually are, and that is statistically significantly different from 0 because it is about 7 times as large as its standard error.

For kicks, here is a way to recover the estimate for just males, using similar code:

In [ ]:
reg3m <- lm(diff ~ 1, 
            data = subset(ah_heights, female == 0)
            )
summary(reg3m)
confint(reg3m)

<h3>4.3. Or redefine the group indicator</h3>

One could also define an indicator variable `male` and run OLS using that instead of `female`. Here is one way to set that up, with the absolute value function `abs()`:

In [ ]:
ah_heights$male = abs(ah_heights$female - 1)
head(ah_heights)

In [ ]:
reg4 <- lm(diff ~ male, data = ah_heights)
summary(reg4)
confint(reg4)

Voila: mirror-image results.

And just for more kicks, note what happens when we try to include BOTH `female` and `male` in a linear model. If there were observations that did not have either `female == 1` or `male == 1` in the data, this would not be a problem, and the constant term would show the average $y$-variable for that other group that was neither male nor female. But here, the constant term is not well defined if we have already included indicators for all gender groups in the data.

To solve this problem, __R__ appears to drop the last indicator listed.

In [ ]:
reg5 <- lm(diff ~ male + female, data = ah_heights)
summary(reg5)
confint(reg5)

An option (literally) is to ask `lm()` to suppress the constant term, like this:

In [ ]:
reg6 <- lm(diff ~ 0 + male + female, data = ah_heights)
summary(reg6)
confint(reg6)

As you can see, this model gives us part of what we want, but perhaps not all. For example, if we wanted to state something about the difference between men and women here, we would need to use `linearHypothesis()` again.

There will likely be something you want to understand that will require a little more work, regardless of how you set up the first model. C'est la statistiques.

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>